# First BigQuery ML models for Taxifare Prediction

In this notebook, we will use BigQuery ML to build our first models for taxifare prediction.

BigQuery ML provides a fast way to build ML models on large structured and semi-structured datasets.

We'll start by creating a dataset to hold all the models we create in BigQuery

In [1]:
!bq mk serverlessml

Dataset 'cloud-training-demos:serverlessml' successfully created.


## Model 1: Raw data

Let's build a model using just the raw data. It's not going to be very good, but sometimes it is good to actually experience this.

The model will take a minute or so to train. When it comes to ML, this is blazing fast.

In [10]:
%%bigquery
CREATE OR REPLACE MODEL serverlessml.model1_rawdata
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg') AS

SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers
FROM `nyc-tlc.yellow.trips`
WHERE MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 100000) = 1

""


Once the training is done, visit the [BigQuery Cloud Console](https://console.cloud.google.com/bigquery) and look at the model that has been trained. Then, come back to this notebook.

Note that BigQuery automatically split the data we gave it, and trained on only a part of the data and used the rest for evaluation. We can look at eval statistics on that held-out data:

In [11]:
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL serverlessml.model1_rawdata)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,6.086824,93.909516,0.32947,4.501958,0.00215,0.002582


Let's report just the error we care about, the Root Mean Squared Error (RMSE)

In [19]:
%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL serverlessml.model1_rawdata)

,rmse
0,9.690692


We told you it was not going to be good!  Recall that our heuristic got 7.42, and our target is $6.

Note that the error is going to depend on the dataset that we evaluate it on.
We can also evaluate the model on our own held-out benchmark/test dataset, but we shouldn't make a habit of this (we want to keep our benchmark dataset as the final evaluation, not make decisions using it all along the way. If we do that, our test dataset won't be truly independent).

In [12]:
%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL serverlessml.model1_rawdata,
(
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers
FROM `nyc-tlc.yellow.trips`
WHERE MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 100000) = 2
AND
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
))

,rmse
0,9.428081


## Model 2: Apply data cleanup

Recall that we did some data cleanup in the previous lab. Let's do those before training.

This is a dataset that we will need quite frequently in this notebook, so let's extract it first.

In [22]:
%%bigquery
CREATE OR REPLACE TABLE serverlessml.cleaned_training_data AS

SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers
FROM `nyc-tlc.yellow.trips`
WHERE MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 100000) = 1
AND
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0

""


In [24]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM serverlessml.cleaned_training_data
LIMIT 0

,fare_amount,pickuplon,pickuplat,dropofflon,dropofflat,passengers


In [ ]:
%%bigquery
CREATE OR REPLACE MODEL serverlessml.model2_cleanup
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg') AS

SELECT
*
FROM
serverlessml.cleaned_training_data

Executing query with job ID: 33095783-a522-4a84-a489-8ba1ae6ba95f
Query executing: 4.03s

In [18]:
%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL serverlessml.model2_cleanup)

,rmse
0,7.919123


## Model 3: More sophisticated models

What if we throw a more sophisticated model? Let's try boosted trees first:

### xgboost

In [ ]:
%%bigquery
-- These model types are in alpha, so they may not work for you yet. This training takes on the order of 15 minutes.
CREATE OR REPLACE MODEL serverlessml.model3a_xgboost
OPTIONS(input_label_cols=['fare_amount'], model_type='boosted_tree_regressor') AS

SELECT
*
FROM
serverlessml.cleaned_training_data

In [27]:
%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL serverlessml.model3a_xgboost)

,rmse
0,32.548259


Ouch!

### DNN
Lately, though, deep neural networks are all the rage. What if we use DNNs instead?

In [ ]:
%%bigquery
-- These model types are in alpha, so they may not work for you yet. This training takes on the order of 15 minutes.
CREATE OR REPLACE MODEL serverlessml.model3b_dnn
OPTIONS(input_label_cols=['fare_amount'], model_type='dnn_regressor', hidden_units=[32, 8]) AS

SELECT
*
FROM
serverlessml.cleaned_training_data

In [25]:
%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL serverlessml.model3b_dnn)

,rmse
0,4.779935


Oh, wow! Later in this sequence of notebooks, we will get to below $4, but this is quite good, for very little work.

In this notebook, we showed you how to use BigQuery ML to quickly build ML models. We will come back to BigQuery ML when we want to experiment with different types of feature engineering. The speed of BigQuery ML is very attractive for development.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.